In [ ]:
%pip install pydub
import numpy as np
import pandas as pd
import pickle
import grid_search as gs
from DBcontrol import init_db

import IPython.display as ipd
import soundfile as sf


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
n_songs = 4
signal_to_noise_ratio = 6  # dB
samples = gs.sample_from_source_audios(n_songs, snr=signal_to_noise_ratio)

grid_viewer = gs.run_grid_search(n_songs=4)

paramset_idx = 0
paramset = grid_viewer.paramset_idx_to_params(paramset_idx)
print(f"params that correspond to paramset_idx = {paramset_idx}")
print(paramset)

paramset_idx 0: 1 / 2
{'constellation_mapping': {'cm_window_size': 10, 'candidates_per_band': 6, 'bands': [[0, 10], [10, 20], [20, 40], [40, 80], [80, 160], [160, 512]]}, 'hashing': {'fanout_t': 100, 'fanout_f': 1500}, 'database': {'library_path': 'sql/library.db', 'use_index': True}}
	snr=3: 1 / 2
	snr=6: 2 / 2
paramset_idx 1: 2 / 2
{'constellation_mapping': {'cm_window_size': 10, 'candidates_per_band': 2, 'bands': [[0, 10], [10, 20], [20, 40], [40, 80], [80, 160], [160, 512]]}, 'hashing': {'fanout_t': 100, 'fanout_f': 1500}, 'database': {'library_path': 'sql/library.db', 'use_index': True}}
	snr=3: 1 / 2
	 exception occured
	snr=6: 2 / 2
	 exception occured
params that correspond to paramset_idx = 0
{'cm_window_size': 10, 'candidates_per_band': 6, 'bands': [(0, 10), (10, 20), (20, 40), (40, 80), (80, 160), (160, 512)], 'fanout_t': 100, 'fanout_f': 1500}


## `paramset 0`, SNR = -3 dB

In [ ]:
#n_songs = 4
#signal_to_noise_ratio = 6  # dB

n_songs = 4  # n_songs=None to use all songs
signal_to_noise_ratio = -3  # dB, negative => more noise than signal
samples = gs.sample_from_source_audios(n_songs, snr=signal_to_noise_ratio)

# initialize database with selected parameter set
# **dict notation:
#         convert {"k1": "v1", "k2: "v2"}
#               -> k1="v1", k2="v2"
gs.set_parameters(**paramset)
init_db(n_songs=n_songs)

proportion_correct, results = gs.perform_recognition_test(
    n_songs=n_songs, samples=samples,
    perform_no_index_searches=True,
    store_microphone_samples=True)

print(f"proportion correct = {proportion_correct}")
print(f"snr = {signal_to_noise_ratio}")
df = pd.DataFrame(results)
df.head()

first_audio_sample = df["microphone_audio"][0]

ipd.display(ipd.Audio(first_audio_sample, rate=11025))
#sf.write("sample1.wav", first_audio_sample, 11025)

proportion correct = 0.6
snr = -3


,ground_truth,prediction,correct,metrics,search_time_with_index,search_time_without_index,microphone_audio
0,1,1,True,"{1: {'std_of_deltaT': 15.800709955196966, 'n_h...",0.002502,1.374245,"[-0.013693006863094812, 0.24533119211236118, 0..."
1,1,1,True,"{1: {'std_of_deltaT': 10.726124861473131, 'n_h...",0.002579,1.349979,"[0.027260447828950804, 0.08172861504929683, 0...."
2,1,1,True,"{1: {'std_of_deltaT': 11.08108245181711, 'n_ha...",0.002387,1.330334,"[-0.5648264675677894, -0.8685998790153552, -1...."
3,1,1,True,"{1: {'std_of_deltaT': 8.899807066008922, 'n_ha...",0.002239,1.331887,"[0.5275023473625762, 0.7203752289992055, 0.834..."
4,1,1,True,"{1: {'std_of_deltaT': 25.800335022107838, 'n_h...",0.002322,1.417573,"[0.8277468471555032, 1.0, 0.6614867534996498, ..."


In [17]:
first_mic_sample_metrics = pd.DataFrame(df["metrics"][0])
first_mic_sample_metrics.head()

,1,4,3,2
std_of_deltaT,15.800710,0.000000,8.243084,0.000000
n_hash_matches,4.000000,1.000000,2.000000,1.000000
n_sample_hashes,296.000000,296.000000,296.000000,296.000000
prop_hash_matches,0.013514,0.003378,0.006757,0.003378
histogram_max_height,2.000000,1.000000,1.000000,1.000000


In [18]:
first_mic_sample_metrics.idxmax(axis=1)

std_of_deltaT           1
n_hash_matches          1
n_sample_hashes         1
prop_hash_matches       1
histogram_max_height    1
dtype: int64

## `paramset 0`, SNR = 10 dB

In [19]:
#n_songs = 4
#signal_to_noise_ratio = 6  # dB

n_songs = 4  # n_songs=None to use all songs
signal_to_noise_ratio = 10  # dB, postive => more signal than noise
samples = gs.sample_from_source_audios(n_songs, snr=signal_to_noise_ratio)

# initialize database with selected parameter set
# **dict notation:
#         convert {"k1": "v1", "k2: "v2"}
#               -> k1="v1", k2="v2"
gs.set_parameters(**paramset)
init_db(n_songs=n_songs)

proportion_correct, results = gs.perform_recognition_test(
    n_songs=n_songs, samples=samples,
    perform_no_index_searches=True,
    store_microphone_samples=True)

print(f"proportion correct = {proportion_correct}")
print(f"snr = {signal_to_noise_ratio}")
df = pd.DataFrame(results)
df.head()

first_audio_sample = df["microphone_audio"][0]

ipd.display(ipd.Audio(first_audio_sample, rate=11025))
#sf.write("sample1.wav", first_audio_sample, 11025)

proportion correct = 0.8
snr = 10


In [6]:
from cm_visualizations import visualize_map_interactive
from DBcontrol import retrieve_song
plastic_beach = retrieve_song(1)["audio_path"]
visualize_map_interactive(plastic_beach)

(513, 1632)
